<H1>The battle of the neighborhoods - Week 1</h1>

<h1>Data :</h1>

<h2>One city will be analyzed in this project: Bogotá City.

We will be using the below datasets for analyzing Bogota city</h2>

<h2>Data 1:</h2>

<body><font size="5"><p align= center*left*right*justify>The capital district of Bogotá has 19 locations. To separate the locations and explore them, we will essentially need a set of data that contains the 19 locations that exist in the district, as well as the latitude and latitude coordinates of each neighborhood.
This set of data exists for free on the web.</p> The link to the data set is <a href="https://es.wikipedia.org/wiki/Anex:Bogot%C3%A1_Localities">https://es.wikipedia.org/wiki/Anex:Bogot%C3%A1_Localities</a> </body>

In [5]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         156 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 conda-forge --> 2019.11.28-py36_0 anaconda
    openssl: 1.1.1d-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0  anaconda


certifi-2019.11.28   | 156 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [6]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.11.28-py36_0 anaconda --> 2019.11.28-py36_0 conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1-h7b6447c_0  anaconda --> 1.1.1d-h516909a_0 conda-forge


certifi-2019.11.28   | 149 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [7]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [8]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

import numpy as np # library to handle data in a vectorized manner

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [9]:
website_url = requests.get("https://es.wikipedia.org/wiki/Anexo:Localidades_de_Bogot%C3%A1").text

In [10]:
soup = BeautifulSoup(website_url,'html5lib')
My_table = soup.find('table',{'class':'sortable wikitable'})
My_table
links = My_table.findAll('a')
links
Localidades = []
for link in links:
    Localidades.append(link.get('title'))
    
print(Localidades)
del Localidades[0:2]
Localidades
df = pd.DataFrame()
df['Localidades'] = Localidades
df
def get_coords_local(localidad, output_as='center'):
    """
    get the bounding box of a locality in WGS84 given its name
    Parameters
    ----------
    localidad : str
        name of the country in english and lowercase
    output_as : 'str
        chose from 'boundingbox' or 'center'. 
         - 'boundingbox' for [latmin, latmax, lonmin, lonmax]
         - 'center' for [latcenter, loncenter]
    Returns
    -------
    output : list
        list with coordinates as str
    """
    # create url
    url = '{0}{1}{2}'.format('http://nominatim.openstreetmap.org/search.php?q=',
                             localidad+', Bogota, Bogota Capital District',
                             '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    # parse response to list
    if output_as == 'boundingbox':
        lst = response[output_as]
        output = [float(i) for i in lst]
    if output_as == 'center':
        lst = [response.get(key) for key in ['lat','lon']]
        output = [float(i) for i in lst]
    return output
df2 = df.copy()

latitudeCln = []
longitudeCln = []
for index, row in df2.iterrows():
    print(row[0])
    lat, long = get_coords_local(localidad=row[0], output_as='center')
    latitudeCln.append(lat)
    longitudeCln.append(long)

df2['Latitude'] = latitudeCln
df2['Longitude'] = longitudeCln
df2

[None, None, 'Usaquén', 'Chapinero', 'Santa Fe (Bogotá)', 'San Cristóbal (Bogotá)', 'Usme', 'Tunjuelito', 'Bosa (Bogotá)', 'Kennedy (Bogotá)', 'Fontibón', 'Engativá', 'Suba', 'Barrios Unidos (Bogotá)', 'Teusaquillo', 'Los Mártires', 'Antonio Nariño (Bogotá)', 'Puente Aranda', 'La Candelaria', 'Rafael Uribe Uribe (Bogotá)', 'Ciudad Bolívar (Bogotá)', 'Sumapaz (Bogotá)']
Usaquén
Chapinero
Santa Fe (Bogotá)
San Cristóbal (Bogotá)
Usme
Tunjuelito
Bosa (Bogotá)
Kennedy (Bogotá)
Fontibón
Engativá
Suba
Barrios Unidos (Bogotá)
Teusaquillo
Los Mártires
Antonio Nariño (Bogotá)
Puente Aranda
La Candelaria
Rafael Uribe Uribe (Bogotá)
Ciudad Bolívar (Bogotá)
Sumapaz (Bogotá)


,Localidades,Latitude,Longitude
0,Usaquén,4.694969,-74.031093
1,Chapinero,4.645377,-74.061943
2,Santa Fe (Bogotá),4.602204,-74.078837
3,San Cristóbal (Bogotá),4.548658,-74.047473
4,Usme,4.508155,-74.114328
5,Tunjuelito,4.562204,-74.127647
6,Bosa (Bogotá),4.609913,-74.184710
7,Kennedy (Bogotá),4.631582,-74.151319
8,Fontibón,4.673327,-74.144732
9,Engativá,4.708695,-74.109643


<h2>Data 2:</h2>

<body><font size="5"><p align= center*left*right*justify>The geographic coordinate data of the city of Bogotá will be used as input for the Foursquare API, which will be used to provide information. We will use the Foursquare API to explore through 4 search categories information on how the target market of Crepes and Waffles is distributed, for this we will take into account the following categories:</p>
<ol>
    <lh> Categories of Foursquare </lh>
<li>Restaurants 4bf58dd8d48988d14e941735</li>
<li>Business center 56aa371be4b08b9a8d573517</li>
<li>Convention Center 4bf58dd8d48988d1ff931735</li>
<li>Offices 4d4b7105d754a06375d81259</li>
    </ol></p></body>

<body><font size="5"><p align= center*left*right*justify>This information can be found at the link:</p>
    <a https://developer.foursquare.com/docs/resources/categoriess">https://developer.foursquare.com/docs/resources/categories</a> 
<p>In this way it will be possible to determine the location and location with the best panorama for the new Crepes and Waffle restaurant.</p></body>